In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os

Mounted at /content/drive


In [3]:
os.chdir('/content/drive/MyDrive/Final Year Project')
print(os.getcwd())

/content/drive/MyDrive/Final Year Project


In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Input
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
import numpy as np

In [5]:
img_path = '/content/drive/MyDrive/Final Year Project/Image Classifier/dataset/training_set/Brain/Brain_13.jpg'
img = image.load_img(img_path, color_mode='rgb', target_size=(224,224))
#display(img)
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)

In [6]:
img.shape

(1, 224, 224, 3)

In [7]:
model_img = VGG16(include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


In [8]:
model_img.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [9]:
img_out_vgg = model_img(img)

In [10]:
out_img = img_out_vgg.numpy()

In [11]:
out_img.shape

(1, 7, 7, 512)

Lower dimensions. Can be improved using methods like PCA

In [12]:
model2_img = Sequential([
  Input(shape=(7,7,512)),
  Flatten(),
  Dense(units=4096, activation='relu'),
  Dense(units=2048, activation='relu'),
  Dense(units=512, activation='relu')                   
])

In [13]:
model2_img.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 4096)              102764544 
                                                                 
 dense_1 (Dense)             (None, 2048)              8390656   
                                                                 
 dense_2 (Dense)             (None, 512)               1049088   
                                                                 
Total params: 112,204,288
Trainable params: 112,204,288
Non-trainable params: 0
_________________________________________________________________


In [15]:
out_img = model2_img(out_img).numpy()

In [16]:
out_img.shape

(1, 512)

out_img is now a 1x512 numpy array representing the image

# Question Classifier

In [17]:
import json

In [18]:
with open('Question Classifier/train.json') as f:
    train_data = json.load(f)
with open('Question Classifier/test.json') as f:
    test_data = json.load(f)
with open('Question Classifier/validate.json') as f:
    valid_data = json.load(f)

In [19]:
train_data[0]

{'answer': 'MRI',
 'answer_type': 'OPEN',
 'base_type': 'vqa',
 'content_type': 'Modality',
 'img_id': 1,
 'img_name': 'xmlab1/source.jpg',
 'location': 'Abdomen',
 'modality': 'MRI',
 'q_lang': 'en',
 'qid': 0,
 'question': 'What modality is used to take this image?',
 'triple': ['vhead', '_', '_']}

In [20]:
answers = {}

In [21]:
new_train_data_oc = {}
new_train_data_qa = {}

i=0

for item in train_data:
  if item['q_lang']=='en' and item['base_type']=='vqa':
    
    answer = item['answer']
    question = item['question']
    ans_type = item['answer_type']

    if ',' in answer or ' and ' in answer:
        continue
    if ans_type == 'CLOSED' and answer not in ('Yes', 'No'):
        continue
    
    new_train_data_oc[question] = 1 if ans_type == 'OPEN' else 0
    
    if answer not in answers:
        answers[answer] = i
        i += 1

    new_train_data_qa[question] = answers[answer]

In [22]:
new_test_data_oc = {}
new_test_data_qa = {}

i=0

for item in test_data:
  if item['q_lang']=='en' and item['base_type']=='vqa':
    
    answer = item['answer']
    question = item['question']
    ans_type = item['answer_type']

    if ',' in answer or ' and ' in answer:
        continue
    if ans_type == 'CLOSED' and answer not in ('Yes', 'No'):
        continue
    
    new_test_data_oc[question] = 1 if ans_type == 'OPEN' else 0
    
    if answer not in answers:
        answers[answer] = i
        i += 1

    new_test_data_qa[question] = answers[answer]

In [23]:
new_valid_data_oc = {}
new_valid_data_qa = {}

i=0

for item in valid_data:
  if item['q_lang']=='en' and item['base_type']=='vqa':
    
    answer = item['answer']
    question = item['question']
    ans_type = item['answer_type']

    if ',' in answer or ' and ' in answer:
        continue
    if ans_type == 'CLOSED' and answer not in ('Yes', 'No'):
        continue
    
    new_valid_data_oc[question] = 1 if ans_type == 'OPEN' else 0
    
    if answer not in answers:
        answers[answer] = i
        i += 1

    new_valid_data_qa[question] = answers[answer]

In [ ]:
for item in new_train_data_qa:
    if new_train_data_qa[item] not in ("Yes", "No"):
        print(item + " " + str(new_train_data_qa[item]))

In [25]:
print(answers)

{'MRI': 0, 'Abdomen': 1, 'T2': 2, 'Yes': 3, 'No': 4, 'Right': 5, 'CT': 6, 'Chest': 7, 'Liver': 8, 'Spinal Cord': 9, 'Lung': 10, 'Lung Cancer': 11, 'None': 12, 'Lower right': 13, 'Heart': 14, 'Not seen': 15, 'X-Ray': 16, 'Effusion': 17, 'Noudle': 18, 'Nodule': 19, 'Atelectasis': 20, 'Cardiomegal': 21, 'Center': 22, 'Pneumonia': 23, 'Cardiomega': 24, 'Both Lung': 25, 'Cardiomegaly': 26, '2': 27, 'Infiltration': 28, 'Mass': 29, 'Both Lungs': 30, 'Pneumothorax': 31, 'Left Lung': 32, 'Liver Cancer': 33, 'Left': 34, '0': 35, 'Top': 36, 'Transverse  Plane': 37, 'Right Lung': 38, 'Left Kidney': 39, 'Right Kidney': 40, '1': 41, 'Spleen': 42, 'Upper Left': 43, 'Stomach': 44, 'Oval': 45, 'Irregular': 46, 'T1': 47, 'Intestine': 48, 'Pulmonary bronchus': 49, 'Symmetrical to the spine': 50, 'Symmetrical to the bone marrow': 51, 'Lower Left': 52, 'Grey circle on the left': 53, 'Gray ball on the left': 54, 'Large Bowel': 55, '4': 56, '5': 57, '3': 58, 'Small Bowel': 59, 'Upper left of spleen': 60, 'Up

In [26]:
print(len(answers))

123


In [27]:
train_questions = list(new_train_data_oc.keys())
test_questions = list(new_test_data_oc.keys())
valid_questions = list(new_valid_data_oc.keys())

train_types = list(new_train_data_oc.values())
test_types = list(new_test_data_oc.values())
valid_types = list(new_valid_data_oc.values())

train_answers = list(new_train_data_qa.values())
test_answers = list(new_test_data_qa.values())
valid_answers = list(new_valid_data_qa.values())

In [28]:
train_questions = train_questions + test_questions[:10] + valid_questions[:10]
train_types = train_types + test_types[:10] + valid_types[:10]

test_types = test_types[10:]
valid_types = valid_types[10:]

In [29]:
test_questions = test_questions[10:]
valid_questions = valid_questions[10:]

In [30]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding

In [31]:
t = Tokenizer()

In [32]:
t.fit_on_texts(train_questions)

In [33]:
vocab_size = len(t.word_index) + 1

In [34]:
print(t.word_index)

{'the': 1, 'in': 2, 'is': 3, 'this': 4, 'image': 5, 'what': 6, 'picture': 7, 'of': 8, 'does': 9, 'on': 10, 'lung': 11, 'where': 12, 'are': 13, 'organ': 14, 'right': 15, 'color': 16, 'left': 17, 'can': 18, 'be': 19, 'observed': 20, 'part': 21, 'located': 22, 'brain': 23, 'which': 24, 'contain': 25, 'lower': 26, 'show': 27, 'shown': 28, 'how': 29, 'many': 30, 'upper': 31, 'body': 32, 'liver': 33, 'there': 34, 'black': 35, 'look': 36, 'shape': 37, 'a': 38, 'abnormal': 39, 'normal': 40, 'effusion': 41, 'disease': 42, 'kidney': 43, 'nodule': 44, 'exist': 45, 'chest': 46, 'top': 47, 'appear': 48, 'do': 49, 'mass': 50, 'healthy': 51, 'abnormalities': 52, 'organs': 53, 'patient': 54, 'cardiomegaly': 55, 'pneumonia': 56, 'tumor': 57, 'spleen': 58, 'kidneys': 59, 'gray': 60, 'cord': 61, 'lobe': 62, 'head': 63, 'have': 64, 'atelectasis': 65, 'spinal': 66, 'to': 67, 'heart': 68, 'white': 69, 'about': 70, 'eyes': 71, 'femoral': 72, 'edema': 73, 'abnormality': 74, 'plane': 75, 'hole': 76, 'mri': 77,

In [35]:
vocab_size

187

In [36]:
encoded = t.texts_to_sequences(train_questions)
print(list(train_questions)[0])
print(encoded)

What modality is used to take this image?
[[6, 84, 3, 97, 67, 103, 4, 5], [24, 21, 8, 1, 32, 9, 4, 5, 98, 67], [6, 3, 1, 120, 121, 2, 4, 5], [9, 1, 7, 25, 33], [9, 1, 7, 25, 43], [9, 1, 7, 25, 58], [12, 3, 1, 33], [6, 3, 1, 104, 14, 2, 1, 7], [9, 1, 7, 25, 11], [9, 1, 7, 25, 68], [3, 1, 11, 51], [6, 3, 1, 122, 14, 2, 1, 5], [6, 105, 13, 106, 2, 1, 7], [9, 1, 11, 36, 39], [9, 1, 11, 36, 40], [9, 58, 36, 39], [12, 3, 1, 58], [13, 34, 52, 2, 4, 5], [9, 4, 5, 36, 39], [9, 1, 11, 36, 51], [12, 3, 13, 1, 74, 22], [29, 30, 59, 13, 34, 2, 4, 5], [3, 1, 58, 51], [3, 1, 11, 151], [13, 1, 59, 51], [9, 4, 5, 36, 40], [6, 3, 1, 14, 10, 1, 17, 8, 1, 15, 43], [3, 1, 33, 51], [6, 3, 1, 14, 10, 1, 17, 85, 8, 4, 5], [29, 30, 59, 2, 4, 5], [6, 3, 1, 14, 10, 1, 47, 8, 1, 32, 2, 4, 5], [6, 3, 1, 123, 75, 8, 4, 5], [6, 3, 1, 14, 22, 2, 1, 26, 15, 86, 8, 1, 32, 2, 4, 5], [6, 3, 1, 124, 14, 2, 4, 5], [6, 3, 1, 14, 22, 2, 1, 26, 15, 86], [6, 3, 1, 14, 10, 1, 26, 15, 85, 8, 4, 5], [6, 14, 3, 1, 35, 76, 2, 1, 31

In [37]:
padded = pad_sequences(encoded, padding='post')
padded = np.array(padded)

In [38]:
embeddings_index = dict()

f = open('Question Classifier/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()

print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [39]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
  embedding_vector = embeddings_index.get(word)
  if word.endswith("'s"):
    embedding_vector = embeddings_index.get(word[:-2])
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector
  else:
    print(word)

cardiomegaly
parotids


In [40]:
##########################################################

In [41]:
from keras.layers import LSTM, Dropout
import tensorflow.keras.utils as ku

In [42]:
model_ques = Sequential([
    Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=padded.shape[1], trainable=False),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    Flatten(),
    Dense(units=512),
    Dropout(0.2)
])

In [43]:
model_ques.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 16, 100)           18700     
                                                                 
 lstm (LSTM)                 (None, 16, 50)            30200     
                                                                 
 dropout (Dropout)           (None, 16, 50)            0         
                                                                 
 lstm_1 (LSTM)               (None, 16, 50)            20200     
                                                                 
 dropout_1 (Dropout)         (None, 16, 50)            0         
                                                                 
 lstm_2 (LSTM)               (None, 16, 50)            20200     
                                                                 
 dropout_2 (Dropout)         (None, 16, 50)           

In [44]:
sample = ["What modality is this picture, ct or mri?"]
sequence = t.texts_to_sequences(sample)
pad_sequence = pad_sequences(sequence, maxlen=padded.shape[1], padding='post')

In [45]:
pad_sequence

array([[  6,  84,   3,   4,   7, 110,  77,   0,   0,   0,   0,   0,   0,
          0,   0,   0]], dtype=int32)

In [46]:
out_ques = model_ques(pad_sequence)

In [47]:
out_ques_num = out_ques.numpy()

In [48]:
out_ques_num.shape

(1, 512)

out_ques_num is a 1x512 numpy array representing the question

In [49]:
joint_img_ques_concat = np.concatenate([out_img, out_ques_num])

In [50]:
joint_img_ques_concat.shape

(2, 512)

joint_img_ques_concat is the concatenated joint representation for the image and question. Can be replaced with element-wise product or attention networks.

In [53]:
classify_model = Sequential([
     Input(shape=(2,512)),
     Flatten(),
     Dense(units=256, activation='relu'),
     Dense(units=123, activation='softmax')
])

In [56]:
image_question_pair = []
answers = []

i=1

for question_image in train_data:
    if question_image['q_lang'] == 'en' and question_image['base_type'] == 'vqa' and i<=100:
        new_train_data[question_image['question']] = (question_image['question'], question_image['answer'])

[{'img_id': 1, 'img_name': 'xmlab1/source.jpg', 'question': 'What modality is used to take this image?', 'answer': 'MRI', 'q_lang': 'en', 'location': 'Abdomen', 'modality': 'MRI', 'answer_type': 'OPEN', 'base_type': 'vqa', 'content_type': 'Modality', 'triple': ['vhead', '_', '_'], 'qid': 0}, {'img_id': 1, 'img_name': 'xmlab1/source.jpg', 'question': 'Which part of the body does this image belong to?', 'answer': 'Abdomen', 'q_lang': 'en', 'location': 'Abdomen', 'modality': 'MRI', 'answer_type': 'OPEN', 'base_type': 'vqa', 'content_type': 'Position', 'triple': ['vhead', '_', '_'], 'qid': 1}, {'img_id': 1, 'img_name': 'xmlab1/source.jpg', 'question': 'What is the mr weighting in this image?', 'answer': 'T2', 'q_lang': 'en', 'location': 'Abdomen', 'modality': 'MRI', 'answer_type': 'OPEN', 'base_type': 'vqa', 'content_type': 'Modality', 'triple': ['vhead', '_', '_'], 'qid': 2}, {'img_id': 1, 'img_name': 'xmlab1/source.jpg', 'question': 'Does the picture contain liver?', 'answer': 'Yes', 'q_

In [ ]:
img_main_path = 'Slake1.0/imgs'
img_path = img_main_path + img_side_path
img = image.load_img(img_path, color_mode='rgb', target_size=(224,224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
out_img = model_img(img).numpy()

question = ['']
sequence = t.texts_to_sequences(question)
pad_sequence = pad_sequences(sequence, maxlen=padded.shape[1], padding='post')
out_ques = model_ques(pad_sequence).numpy()

